<a href="https://colab.research.google.com/github/bogdanlalu/tensorflow/blob/master/NLP/03_Binary_Classifier_using_Embeddigs_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<a href="https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20In%20Practice/Course%203%20-%20NLP/Course%203%20-%20Week%202%20-%20Lesson%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
print(tf.__version__)

# !pip install -q tensorflow-datasets

2.2.0


In [0]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteEOCMDI/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteEOCMDI/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteEOCMDI/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [0]:
!ls -lh /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0/

total 132M
-rw-r--r-- 1 root root 3.3K May 13 08:57 dataset_info.json
-rw-r--r-- 1 root root  33M May 13 08:57 imdb_reviews-test.tfrecord-00000-of-00001
-rw-r--r-- 1 root root  33M May 13 08:57 imdb_reviews-train.tfrecord-00000-of-00001
-rw-r--r-- 1 root root  67M May 13 08:57 imdb_reviews-unsupervised.tfrecord-00000-of-00001
-rw-r--r-- 1 root root    8 May 13 08:57 label.labels.txt


In [0]:
imdb

{'test': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>,
 'train': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>,
 'unsupervised': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>}

In [0]:
imdb['train']

<DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>

In [0]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [0]:
print("training sentences: ", len(training_sentences))
print("training labels: ", len(training_labels))

print("testing sentences: ", len(testing_sentences))
print("testing labels: ", len(testing_labels))

training sentences:  25000
training labels:  25000
testing sentences:  25000
testing labels:  25000


In [0]:
print(training_sentences[0])

b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."


In [0]:
print(training_labels[0])

0


# Original padding

In [0]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)


In [0]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

b'this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received ' ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
b'This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.'


In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),  # here we could also use GlobalAveragePooling1D() instead of Flatten()
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [0]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 5s 7ms/step - loss: 0.4905 - accuracy: 0.7416 - val_loss: 0.3820 - val_accuracy: 0.8283
Epoch 2/10
782/782 [==============================] - 5s 7ms/step - loss: 0.2395 - accuracy: 0.9087 - val_loss: 0.3783 - val_accuracy: 0.8344
Epoch 3/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0929 - accuracy: 0.9759 - val_loss: 0.4461 - val_accuracy: 0.8282
Epoch 4/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0215 - accuracy: 0.9974 - val_loss: 0.5205 - val_accuracy: 0.8272
Epoch 5/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0049 - accuracy: 0.9998 - val_loss: 0.5847 - val_accuracy: 0.8275
Epoch 6/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.6332 - val_accuracy: 0.8293
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 8.1797e-04 - accuracy: 1.0000 - val_loss: 0.6772 - val_accuracy: 0.8296
Ep

# Revised padding (my approach)

 - In this approach we set padding and truncating to 'post' in both the training and testing data

In [0]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [0]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

b'this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received ' ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
b'This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.'


In [0]:
tokenizer.sequences_to_texts(padded[3:4])

["b'this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received ' <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV>"]

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_2 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [0]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 5s 7ms/step - loss: 0.5003 - accuracy: 0.7348 - val_loss: 0.3801 - val_accuracy: 0.8273
Epoch 2/10
782/782 [==============================] - 5s 7ms/step - loss: 0.2355 - accuracy: 0.9091 - val_loss: 0.4142 - val_accuracy: 0.8179
Epoch 3/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0818 - accuracy: 0.9799 - val_loss: 0.5116 - val_accuracy: 0.8067
Epoch 4/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0197 - accuracy: 0.9975 - val_loss: 0.6179 - val_accuracy: 0.7997
Epoch 5/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0050 - accuracy: 0.9998 - val_loss: 0.6757 - val_accuracy: 0.8046
Epoch 6/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.7344 - val_accuracy: 0.8065
Epoch 7/10
782/782 [==============================] - 5s 7ms/step - loss: 8.0618e-04 - accuracy: 1.0000 - val_loss: 0.7846 - val_accuracy: 0.8069
Ep

# --- End revised padding

# Look at embeddings

In [0]:
e = model.layers[0]. # Get the first layer which is the Embedding layer
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [0]:
print(weights[0])

[ 0.0560945  -0.09518651 -0.01271305  0.02725973 -0.03292966 -0.01720801
  0.00031776  0.00361164  0.06846526 -0.01598965 -0.00012839 -0.01235148
 -0.02380612 -0.07581057 -0.02816241 -0.03491032]


Save the tokens in the vocabulary and the corresponding weights to 2 files which can be uploaded to https://projector.tensorflow.org/ to visualise 
the embeddings 

In [0]:
#Original code without context manager
# |import io

# out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
# out_m = io.open('meta.tsv', 'w', encoding='utf-8')
# for word_num in range(1, vocab_size):
#   word = reverse_word_index[word_num]
#   embeddings = weights[word_num]
#   out_m.write(word + "\n")
#   out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
# out_v.close()
# out_m.close()

In [0]:
# My version og the above code using context managers

with open('meta.tsv', 'w', encoding='utf-8') as f:
  for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    f.write(word + "\n")

with open('vecs.tsv', 'w', encoding='utf-8') as f:
  # I think weights[0] is skipped in the loop since it might represent  
  # the weights of the padding token(0). 
  # Therfore, weights[1] aligns with reverse_word_index[1] and so on
  for word_num in range(1, vocab_size):             
    embeddings = weights[word_num]
    f.write('\t'.join([str(x) for x in embeddings]) + '\n')

In [0]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [0]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

[[11], [], [1431], [966], [4], [1537], [1537], [4715], [], [790], [2019], [11], [2929], [2184], [], [790], [2019], [11], [579], [], [11], [579], [], [4], [1782], [4], [4517], [11], [2929], [1275], [], [], [2019], [1003], [2929], [966], [579], [790], []]


In [0]:
sentences_list = ["I really think this is amazing. honest.", "Tomorrow is Sunday"]
sequence = tokenizer.texts_to_sequences(sentences_list)
print(sequence)

[[11, 68, 105, 12, 7, 491, 1216], [5529, 7, 2705]]
